In [39]:
import pathlib
import importlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Tuple
from scipy import stats
import dexplot as dxp
import scipy.stats as ss
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
DATA_PATH = pathlib.Path('./data')
RESULTS_PATH = pathlib.Path('./results')

In [4]:
df = pd.read_csv(DATA_PATH / 'accepted_2007_to_2018Q4.csv')

In [5]:
keep_list = ['addr_state', 'annual_inc', 'application_type', 
             'dti', 'earliest_cr_line', 'emp_length', 
             'emp_title', 'fico_range_high', 'fico_range_low', 
             'grade', 'home_ownership', 'id', 'initial_list_status', 
             'installment', 'int_rate', 'issue_d', 'loan_amnt', 
             'mort_acc', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 
             'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 
             'title', 'total_acc', 'verification_status', 'zip_code', 'loan_status']
keep_list.sort()
df = df[keep_list]

In [6]:
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

In [7]:
df['target'] = 1
df['target'] = (df['target'].where(df['loan_status'] == 'Charged Off', 0))
df = df.drop('loan_status', axis=1)

In [8]:
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

In [9]:
dates_cols = ['earliest_cr_line', 'issue_d']
for c in dates_cols:
    df[c] = pd.to_datetime(df[c], format='%b-%Y')

In [10]:
cat_cols = df.select_dtypes('object').columns

# Preprocessing

Let's start with some preprocessing now that we have some understanding of the data.

We need to start filling the missing values.

For numerical columns, for simplicity, we are going to fill missing values with the median.
For categorical columns, a simple approach would be to fill the missing values with the most common observation.

Keep in mind that these approaches are just to keep things simple. If one thinks that more time should be spent on preprocessing, they could go with fancier methods, including:
- using other ML models to come up with values for the missing 

When filling the missing values, it is important that we calculate this value using the training set only, and then use the same value for the test set
so as to avoid data leakage from the training set to the test set.

In [15]:
years = [2013, 2014, 2015, 2016]
train_years = years[0:3]
test_years = years[3:]
train_df = df[df['issue_d'].dt.year.isin(train_years)]
test_df = df[df['issue_d'].dt.year.isin(test_years)]

# Filling Missing Values

In [16]:
print("Missing Percentage")
for col in df.columns:
    missing_percentage = 100 - 100*df[col].count()/len(df)
    if missing_percentage > 0:
        print(col + " " + str())
        print(missing_percentage)
        print(df[col].dtype)

Missing Percentage
dti 
0.027800283949417803
float64
emp_length 
5.835903992388367
object
emp_title 
6.376597215511666
object
mort_acc 
3.5145059503014124
float64
pub_rec_bankruptcies 
0.05180962008756751
float64
revol_util 
0.0637027896915896
float64
title 
1.23830195270979
object
zip_code 
7.433231002096363e-05
object


A simple approach for numerical features: Fill with the median

For categorical features we can fill with the most frequent values

In [31]:
def preprocessing(df, num_cols, cat_cols):

    for col in num_cols:
        median = df[col].median()
        df[col] = df[col].fillna(median)

    for col in cat_cols:
        most_common = df[col].mode()[0]
        df[col] = df[col].fillna(most_common)
    
    return df

# Remark: the training and test medians might not be exactly the same, so the values filled would be different
# This also applies to the most common observation, which may be different in the test set

In [ ]:
SCALERS = {}
LABEL_ENCODERS = {}

In [ ]:
def feature_engineering(df, num_cols, cat_cols, log_cols, is_train=True):
    ''' This function will be used to take the log of our features, scale our features, encodings, etc.'''

    # taking the log of features in log_cols
    # a problem is that at least one feature has neg values, so we are going to drop those rows
    df = df[df['dti'] > -1]

    for col in log_cols:
        df[col] = np.log1p(df[col])
    
    # not every model requires feature scaling but some do
    # let's scale the numerical features with robustscaler
    for col in num_cols:
        scaler = RobustScaler()
        if is_train:
            scaler.fit_transform(df[col])
            SCALERS[col] = scaler
        else:
            SCALERS[col].transform(df[col])


     # TODO: feature engineering for emp_length

     cat_cols.remove('emp_length')




    for col in cat_cols:
        scaler = LabelEncoder()
        if is_train:
            scaler.fit_transform(df[col])
            SCALERS[col] = scaler
        else:
            SCALERS[col].transform(df[col])

In [58]:
le = LabelEncoder()

le.fit(train_df['addr_state'])

LabelEncoder()

In [60]:
le.transform(df['grade'].head())

ValueError: y contains previously unseen labels: 'C'

For categorical features, we can fill with the most common value.

In [30]:
log_cols = [
    'annual income',
    'dti',
    'installment',
    'int_rate',
    'loan_amnt',
    'open_acc',
    'revol_bal',
    'revol_util',
    'total_acc'
]

0    Teacher
dtype: object

In [37]:
df['mort_acc'].value_counts()

0.0     523837
1.0     226135
2.0     188941
3.0     139381
4.0      94920
5.0      57632
6.0      32539
7.0      16850
8.0       8378
9.0       4285
10.0      2197
11.0      1206
12.0       640
13.0       356
14.0       244
15.0       132
16.0        93
17.0        61
18.0        48
19.0        29
20.0        25
24.0        16
21.0        14
22.0        14
23.0        10
25.0         9
27.0         8
26.0         6
28.0         4
29.0         4
34.0         3
30.0         2
32.0         2
37.0         2
31.0         2
51.0         1
47.0         1
36.0         1
35.0         1
Name: mort_acc, dtype: int64

In [34]:
np.log1p(df[df['dti']==-1])

TypeError: loop of ufunc does not support argument 0 of type str which has no callable log1p method

In [35]:
df.min()

addr_state                               AK
annual_inc                                0
application_type                 Individual
dti                                      -1
earliest_cr_line        1934-04-01 00:00:00
fico_range_high                         629
fico_range_low                          625
grade                                     A
home_ownership                          ANY
initial_list_status                       f
installment                            4.93
int_rate                               5.31
issue_d                 2007-06-01 00:00:00
loan_amnt                               500
mort_acc                                  0
open_acc                                  0
pub_rec                                   0
pub_rec_bankruptcies                      0
purpose                                 car
revol_bal                                 0
revol_util                                0
sub_grade                                A1
term                            